# Нейронка на gpt-3 для векторизации описания фильмов

# import

In [53]:
from Database.Database import DataFunFilm
import pandas as pd
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader as api
import gensim
import re
import numpy as np

# Загрузка базы данных

In [3]:
db = DataFunFilm()
db.getTableDF()

,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits
0,1,1043658,tt4202274,Шоу Грязного Фрэнка,The Filthy Frank Show,https://kinopoiskapiunofficial.tech/images/pos...,9.7,32929,NaN,0,2011.0,12.0,фэнтези комедия музыка короткометражка,https://www.kinopoisk.ru/series/1043658/,None,Шоу повествует о героической борьбе сэра Фрэнс...,9.3,None,None
1,2,1309325,None,Космос,,https://kinopoiskapiunofficial.tech/images/pos...,NaN,0,NaN,0,2019.0,NaN,документальный,https://www.kinopoisk.ru/series/1309325/,None,Астрофизик Нил Деграсс Тайсон встает за штурва...,9.3,Нил Деграсс Тайсон отправляется в мысленное пу...,age18
2,3,77044,tt0108778,Друзья,Friends,https://kinopoiskapiunofficial.tech/images/pos...,9.0,937875,NaN,0,1994.0,22.0,мелодрама комедия,https://www.kinopoisk.ru/series/77044/,Everybody needs Friends!,"Шестеро друзей – Рейчел, Моника, Фиби, Джоуи, ...",9.2,"Шестеро друзей, две квартиры и много юмора. Си...",age16
3,4,972889,tt4926348,BadComedian,None,https://kinopoiskapiunofficial.tech/images/pos...,7.7,793,NaN,0,2011.0,NaN,комедия,https://www.kinopoisk.ru/series/972889/,None,"Юмористическое шоу, высмеивающее плохое кино. ...",9.2,None,age18
4,5,674243,None,Счастливые люди,None,https://kinopoiskapiunofficial.tech/images/pos...,8.8,120,NaN,0,2008.0,55.0,документальный,https://www.kinopoisk.ru/series/674243/,None,Документальный фильм о трудах и днях охотников...,9.2,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66752,66753,83307,tt0041640,Мари из порта,La Marie du port,https://kinopoiskapiunofficial.tech/images/pos...,6.7,395,NaN,0,1950.0,88.0,драма мелодрама,https://www.kinopoisk.ru/film/83307/,None,"Анри Шателяру уже за сорок, он владеет рестора...",7.0,None,None
66753,66754,219369,tt0071859,Путешественник,Mossafer,https://kinopoiskapiunofficial.tech/images/pos...,7.5,1695,NaN,0,1974.0,83.0,драма,https://www.kinopoisk.ru/film/219369/,None,Парнишке-школьнику по имени Кассем из маленько...,7.0,None,None
66754,66755,47614,tt0211372,Сын француза,Le fils du Français,https://kinopoiskapiunofficial.tech/images/pos...,5.0,282,NaN,0,1999.0,107.0,приключения комедия семейный,https://www.kinopoisk.ru/film/47614/,None,У Бенжамена 9 лет нет больше мамы. Он хочет на...,7.0,None,None
66755,66756,977755,tt9378362,Звоните ДиКаприо!,None,https://kinopoiskapiunofficial.tech/images/pos...,6.7,496,NaN,0,2018.0,48.0,драма комедия,https://www.kinopoisk.ru/series/977755/,Что останется после тебя?,Егор Румянцев – звезда популярного сериала про...,7.0,None,age18


In [4]:
table = db.getTableDF()
description = db.remakeDescriprionToArray()
print(description[0])

['шоу', 'повествует', 'о', 'героической', 'борьбе', 'сэра', 'фрэнсиса', 'грязного', 'и', 'его', 'верных', 'друзей', 'против', 'зловещего', 'тёмного', 'бога', 'в', 'сюрреалистической', '«омнивселенной».']


In [ ]:
#print(model.wv.key_to_index)
words_trainded = model.wv.most_similar(positive=words[0], topn=3)
for word in words_trainded:
  print(word)
print()

In [75]:
import gensim.downloader as api
rus_words = api.load('word2vec-ruscorpora-300')
rus_words.key_to_index

{'весь_DET': 0,
 'человек_NOUN': 1,
 'мочь_VERB': 2,
 'год_NOUN': 3,
 'сказать_VERB': 4,
 'время_NOUN': 5,
 'говорить_VERB': 6,
 'становиться_VERB': 7,
 'знать_VERB': 8,
 'самый_DET': 9,
 'дело_NOUN': 10,
 'день_NOUN': 11,
 'жизнь_NOUN': 12,
 'рука_NOUN': 13,
 'очень_ADV': 14,
 'первый_ADJ': 15,
 'давать_VERB': 16,
 'новый_ADJ': 17,
 'слово_NOUN': 18,
 'иметь_VERB': 19,
 'большой_ADJ': 20,
 'идти_VERB': 21,
 'глаз_NOUN': 22,
 'место_NOUN': 23,
 'лицо_NOUN': 24,
 'видеть_VERB': 25,
 'хотеть_VERB': 26,
 'понимать_VERB': 27,
 'должный_ADJ': 28,
 'работа_NOUN': 29,
 'каждый_DET': 30,
 'друг_NOUN': 31,
 'голова_NOUN': 32,
 'дом_NOUN': 33,
 'оставаться_VERB': 34,
 'сторона_NOUN': 35,
 'начинать_VERB': 36,
 'думать_VERB': 37,
 'хорошо_ADV': 38,
 'жить_VERB': 39,
 'стоять_VERB': 40,
 'спрашивать_VERB': 41,
 'сделать_VERB': 42,
 'выходить_VERB': 43,
 'последний_ADJ': 44,
 'русский_ADJ': 45,
 'сила_NOUN': 46,
 'получать_VERB': 47,
 'какой-то_DET': 48,
 'хороший_ADJ': 49,
 'случай_NOUN': 50,
 'во

In [76]:
def getWordVec(n, vectors, word):
  words_vec.append(rus_words[word])
  n += 1
  vectors += words_vec
  return n, vectors

In [77]:
sent_vec = []
n = 0
for i in range(100):
  words_vec = []
  vectors = []
  for word in description[i]:
    if word + "_NOUN" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_NOUN")
    if word + "_VERB" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_VERB")
    if word + "_DET" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_DET")
    if word + "_ADJ" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_ADJ")
  for j in range(len(vectors)):
    sent_vec.append(vectors[j] / n)

In [78]:
def getNumberMinimum(arr, arrmin):
  number = 0
  for i in range(len(arr)):
    if arr[i] < arrmin[i]:
      number += 1

  return number

In [79]:
def compareDescriptions(number):
  min = number - 1
  min_vec = 10000
  for i in range(1000):
    if (i != number):
      iter = np.abs(sent_vec[number] - sent_vec[i])
      minimum = np.abs(sent_vec[number] - sent_vec[min])
      if min_vec < getNumberMinimum(iter, minimum):
        min = i
        min_vec = getNumberMinimum(iter, minimum)
  return min

In [80]:
number = 500

min = compareDescriptions(number)

print(description[min])
print(description[number])

['праздник', 'всевозможных', 'овощных', 'блюд,', 'побуждающих', 'и', 'мясоедов', 'и', 'вегетарианцев', 'всегда', 'ставить', 'перед', 'собой', 'на', 'стол', 'овощи!', 'джейми', 'полагает,', 'что', 'самое', 'время', 'перестать', 'думать', 'об', 'овощах,', 'как', 'о', 'дополнении', 'к', 'основной', 'еде,', '—', 'овощи', 'способны', 'сами', 'стать', 'героем', 'шоу,', 'и', 'джейми', 'готов', 'показать', 'нам', 'как.', 'джейми', 'знает,', 'что', 'многие', 'из', 'нас', 'хотят', 'есть', 'поменьше', 'мяса', '—', 'кто', 'по', 'соображениям', 'здоровья,', 'кто', 'из-за', 'денег,', 'а', 'кто-то', 'из', 'гуманных', 'соображений.', 'все', 'мы', 'медленно', 'движемся', 'в', 'этом', 'направлении', '—', 'есть', 'больше', 'овощей', 'и', 'меньше', 'мяса.']
['трогательная', 'история', 'о', 'супружеской', 'чете', 'мальхотра.', 'они', 'посвятили', 'жизнь,', 'чтобы', 'вырастить', 'четырех', 'сыновей,', 'и', 'как', 'и', 'другие', 'родители', 'рассчитывали,', 'что', 'сыновья', 'станут', 'поддержкой', 'и', 'опо

In [94]:
def getWordsFromDescription(sentence):
  words = []
  for word in sentence:
    if word + "_NOUN" in rus_words:
      words.append(word + "_NOUN")
    elif word + "_VERB" in rus_words:
      words.append(word + "_VERB")
    elif word + "_DET" in rus_words:
      words.append(word + "_DET")
    elif word + "_ADJ" in rus_words:
      words.append(word + "_ADJ")
  return words

In [95]:
def anotherCompareAlgorithm(n, description):
  sentence = getWordsFromDescription(description[n])
  sentence_min = getWordsFromDescription(description[n - 1])
  min_similarity = rus_words.wmdistance(sentence, sentence_min)
  min_num = n - 1
  for i in range(len(description)):
    if i != n:
      new_sentence = getWordsFromDescription(description[i])
      similarity = rus_words.wmdistance(sentence, new_sentence)
      if similarity < min_similarity:
        min_similarity = similarity
        min_num = i
  return min_num

In [96]:

number = 500

min = anotherCompareAlgorithm(number, description)

print(description[min])
print(description[number])

['романтическая', 'история', 'о', 'двух', 'студентах,', 'которые', 'являются', 'представителями', 'абсолютно', 'разных', 'культур.', 'криш', 'родом', 'из', 'дели,', 'а', 'ананья', 'из', 'ченнаи.', 'они', 'решают', 'пожениться,', 'но', 'только', 'если', 'их', 'родители', 'дадут', 'на', 'это', 'согласие...']
['трогательная', 'история', 'о', 'супружеской', 'чете', 'мальхотра.', 'они', 'посвятили', 'жизнь,', 'чтобы', 'вырастить', 'четырех', 'сыновей,', 'и', 'как', 'и', 'другие', 'родители', 'рассчитывали,', 'что', 'сыновья', 'станут', 'поддержкой', 'и', 'опорой', 'на', 'старости', 'лет,', 'но', 'на', 'деле', 'все', 'вышло', 'иначе.']
